# Build a Chatbot

LangChain v0.3 릴리스부터는 LangChain 사용자들이 memory를 새로운 LangChain 애플리케이션에 통합하기 위해 LangGraph 지속성(LangGraph Persistence)을 사용하는 것이 권장됩니다.

이미 RunnableWithMessageHistory 또는 BaseChatMessageHistory에 의존하고 있는 코드라면 아무 변경도 필요하지 않습니다. 이 기능은 간단한 채팅 애플리케이션에서 계속 사용할 수 있으며, RunnableWithMessageHistory를 사용하는 코드는 기대한 대로 계속 작동할 것입니다.

### 개요 (Overview)
LLM을 활용한 챗봇 설계 및 구현 예제를 살펴보겠습니다.
이 챗봇은 대화를 나누고 이전 상호작용을 기억할 수 있습니다.

이 챗봇은 언어 모델만 사용하여 대화를 진행합니다.
다음과 같은 관련 개념도 확인할 수 있습니다:

- Conversational RAG: 외부 데이터 소스를 활용한 챗봇
- 에이전트(Agents): 액션을 수행할 수 있는 챗봇


In [1]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai \
# python-dotenv \
# langgraph

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# LangChain 추적(Tracing) 설정 활성화
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

먼저 모델을 직접 사용해 봅니다. `ChatModel`은 LangChain의 **"Runnable"** 인스턴스이며, 이는 표준화된 인터페이스를 통해 상호작용할 수 있음을 의미합니다.  

모델을 간단하게 호출하려면 `.invoke` 메서드에 **메시지 목록**을 전달하면 됩니다.

In [5]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="안녕! 나는 길동이야.")])

AIMessage(content='안녕하세요, 길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 16, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-278c10c1-2f5a-460e-9539-68bf4e9d1242-0', usage_metadata={'input_tokens': 16, 'output_tokens': 21, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

모델 자체는 **상태(state)**라는 개념을 가지고 있지 않습니다. 예를 들어, 후속 질문을 하면:

In [6]:
model.invoke([HumanMessage(content="내 이름이 뭐라고 했지?")])

AIMessage(content='죄송하지만, 제가 당신의 이름을 알고 있지 않습니다. 당신의 이름을 알려주시면 기억할 수 있습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 15, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-9c0e3817-59b3-4bb7-9c77-5fcf5f3ff6db-0', usage_metadata={'input_tokens': 15, 'output_tokens': 27, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="안녕! 나는 길동이야."),
        AIMessage(content="안녕하세요, 길동님. 무엇을 도와 드릴까요?"),
        HumanMessage(content="내 이름이 뭐라고 했지?"),
    ]
)

AIMessage(content='당신의 이름은 길동이라고 하셨습니다. 맞나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 47, 'total_tokens': 63, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-3b628d3d-b1b7-48a2-8ca9-a215ddd78708-0', usage_metadata={'input_tokens': 47, 'output_tokens': 16, 'total_tokens': 63, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

이제 좋은 응답을 받는 것을 확인할 수 있습니다!  

이것이 챗봇이 **대화형 상호작용**을 할 수 있는 기본 아이디어입니다.  
그렇다면, 이를 가장 효과적으로 구현하는 방법은 무엇일까요?

## **메시지 지속성 (Message Persistence)**  

**LangGraph**는 **내장된 지속성 계층(persistence layer)**을 구현하여 **여러 번의 대화(turns)**를 지원하는 챗 애플리케이션에 이상적입니다.  

챗 모델을 간단한 LangGraph 애플리케이션으로 감싸면 **메시지 기록을 자동으로 저장(persist)**할 수 있어, **다중 턴(multi-turn)** 애플리케이션 개발이 훨씬 더 간편해집니다.  

LangGraph에는 **간단한 인메모리 체크포인터(in-memory checkpointer)**가 포함되어 있으며, 아래 예제에서 이를 사용합니다.  

In [8]:
from langgraph.checkpoint.memory import MemorySaver  # 인메모리 체크포인트를 위한 MemorySaver 임포트
from langgraph.graph import START, MessagesState, StateGraph  # 그래프 시작 지점, 메시지 상태, 그래프 정의를 위한 클래스 임포트

# 새로운 그래프 정의
# 메시지 상태(MessagesState)를 상태 스키마(state_schema)로 사용하여 워크플로우를 정의
workflow = StateGraph(state_schema=MessagesState)

# 모델을 호출하는 함수 정의
def call_model(state: MessagesState):
    """
    LLM(언어 모델)에 메시지를 전달하고 응답을 받습니다.

    Args:
        state (MessagesState): 현재 대화 상태 (이전 메시지 포함)

    Returns:
        dict: 모델의 응답을 포함한 새로운 상태
    """
    response = model.invoke(state["messages"])  # 상태에서 메시지를 추출하고 모델에 전달
    return {"messages": response}                        # 응답을 딕셔너리 형태로 반환


# 그래프 노드 및 엣지 설정
# START 지점에서 "model" 노드로 이동하도록 엣지를 추가합니다.
workflow.add_edge(START, "model")

# "model" 노드에 call_model 함수를 연결합니다.
workflow.add_node("model", call_model)

# 메모리 추가 (메시지 기록 저장)
memory = MemorySaver()  # 인메모리 체크포인트 저장소 생성

# 워크플로우를 메모리 체크포인트와 함께 컴파일합니다.
app = workflow.compile(checkpointer=memory)

이제 매번 `runnable` 객체에 전달할 **`config`**를 생성해야 합니다.  

이 **`config`**에는 입력에 직접 포함되지는 않지만 유용한 정보가 포함됩니다.  이번 경우, **`thread_id`**를 포함할 것 입니다.  

다음과 같은 형태여야 합니다:

In [9]:
config = {"configurable": {"thread_id": "abc123"}}

이렇게 하면 하나의 애플리케이션에서 **여러 대화 스레드(thread)**를 지원할 수 있습니다.  

여러 사용자가 애플리케이션을 동시에 사용할 때 흔히 필요한 기능입니다.  

이제 애플리케이션을 호출할 수 있습니다:

In [10]:
query = "안녕! 내 이름은 길동이야."

# 입력 메시지를 리스트 형태로 정의
# 이는 대화의 첫 번째 메시지가 됩니다.
input_messages = [HumanMessage(query)]

# 'app.invoke'를 사용하여 애플리케이션을 호출
# 첫 번째 매개변수: 입력 메시지
# 두 번째 매개변수: 추가적인 설정(config) - 예: thread_id 등
output = app.invoke({"messages": input_messages}, config)

# 상태(state)에 있는 모든 메시지 중 마지막 메시지 출력
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

안녕, 길동아! 만나서 반가워. 어떻게 도와줄까?


In [11]:
query = "내 이름이 뭐라고 했지?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

너의 이름은 길동이야!


이제 우리의 챗봇은 우리에 대한 정보를 기억합니다.  

`config`에서 다른 **`thread_id`**를 참조하도록 변경하면, 챗봇이 **새로운 대화**를 시작하는 것을 확인할 수 있습니다.

In [12]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

죄송하지만, 당신의 이름을 알 수 있는 정보는 없습니다. 당신의 이름을 말씀해주시면 그에 맞춰 대화할 수 있습니다!


그러나 우리는 항상 **원래의 대화로 돌아갈 수 있습니다** (데이터베이스에 대화를 **저장(persisting)**하고 있기 때문입니다).

In [13]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

너의 이름은 길동이야!


이렇게 하면 챗봇이 여러 사용자와 동시에 대화를 할 수 있습니다.

### 비동기(async) 지원
비동기 지원을 위해 call_model 노드를 **비동기 함수(async function)**로 업데이트하고, 애플리케이션을 호출할 때 .ainvoke를 사용하세요:

```python
# 노드의 비동기 함수:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

# 이전과 동일하게 그래프 정의:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# 비동기 호출:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

```

지금까지 우리는 모델 주위에 간단한 **지속성 계층(persistence layer)**을 추가한 것에 불과합니다. 이제 **프롬프트 템플릿(prompt template)**을 추가하여 챗봇을 더 복잡하고 개인화된 형태로 만들어 봅시다.

---

## ** 프롬프트 템플릿 (Prompt Templates)**  

**프롬프트 템플릿**은 **원시 사용자 입력(raw user input)**을 LLM이 처리할 수 있는 형식으로 변환하는 데 도움을 줍니다.   

1. 먼저, **시스템 메시지(system message)**를 추가하여 **사용자 정의 지침(custom instructions)**을 포함시킵니다. (여전히 메시지를 입력으로 사용)  
2. 다음으로, 메시지 외에 **더 많은 입력 정보**를 추가합니다.  

---

### ** 시스템 메시지(System Message) 추가하기**  

시스템 메시지를 추가하기 위해 **`ChatPromptTemplate`**을 생성합니다.  
여기서는 모든 메시지를 전달하기 위해 **`MessagesPlaceholder`**를 사용할 것입니다.  

이렇게 하면 LLM에 전달되는 입력이 더 구조화되고, 챗봇의 동작을 더 정교하게 제어할 수 있습니다.  

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# LLM이 사용자 입력을 더 잘 처리할 수 있도록 프롬프트 템플릿을 설정합니다.
prompt_template = ChatPromptTemplate.from_messages(
    [
        # LLM의 동작 방식을 정의하는 지침
        (
            "system",
            "당신은 친구 처럼 말합니다. 모든 질문에 최선을 다해 대답하세요.",
        ),

        # Messages Placeholder - 이전 대화 메시지들을 전달합
        MessagesPlaceholder(variable_name="messages"),
    ]
)

이제 이 템플릿을 통합하여 애플리케이션을 업데이트할 수 있습니다.

In [15]:
from langgraph.graph import START, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver

# MessagesState를 상태 스키마로 사용하여 워크플로우를 정의합니다.
workflow = StateGraph(state_schema=MessagesState)

# 모델 호출 함수 정의
def call_model(state: MessagesState):
    """
    현재 상태(State)를 기반으로 프롬프트를 생성하고 모델을 호출합니다.

    Args:
        state (MessagesState): 현재 대화 상태(이전 메시지 포함)

    Returns:
        dict: 모델의 응답을 포함한 새로운 상태
    """
    # 프롬프트 템플릿을 사용하여 상태(state)로부터 프롬프트를 생성합니다.
    prompt = prompt_template.invoke(state)

    # 생성된 프롬프트를 모델에 전달하고 응답을 받습니다.
    response = model.invoke(prompt)

    # 응답을 딕셔너리 형태로 반환합니다.
    return {"messages": response}

# START에서 "model" 노드로 이동하도록 엣지 정의
workflow.add_edge(START, "model")

# "model" 노드에 call_model 함수를 연결
workflow.add_node("model", call_model)

# MemorySaver를 사용하여 상태 및 메시지 기록을 저장합니다.
memory = MemorySaver()

# 그래프를 메모리 체크포인트와 함께 컴파일합니다.
app = workflow.compile(checkpointer=memory)

같은 방식으로 응용 프로그램을 호출합니다:

In [16]:
# 'configurable' 키를 사용하여 추가적인 설정 값을 전달합니다.
# 여기서는 'thread_id'를 사용하여 특정 대화 스레드를 식별합니다.
config = {"configurable": {"thread_id": "abc345"}}

query = "AI 군, 안녕."

# 메시지 목록에 사용자 메시지 추가
input_messages = [HumanMessage(query)]

# 애플리케이션 호출
# 메시지 상태(State), 설정(config) 전달
output = app.invoke({"messages": input_messages}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

안녕하세요! 어떻게 지내고 계신가요? 도움이 필요하시면 언제든지 말씀해 주세요!


In [17]:
query = "내 이름이 뭐지?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

죄송하지만, 당신의 이름을 모르겠어요. 이름을 알려주시면 좋을 것 같아요! 어떻게 불러드릴까요?


프롬프트를 조금 더 복잡하게 만들어 봅니다. 프롬프트 템플릿이 이제 다음과 같다고 가정해 봅니다.

In [18]:
prompt_template = ChatPromptTemplate(
    [
        (
            "system",
            "당신은 도움이 되는 조수입니다. {language}로 최선을 다해 모든 질문에 답하세요..",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

우리는 프롬프트에 새로운 **`language`** 입력을 추가했습니다.  

이제 우리의 애플리케이션은 두 개의 매개변수를 가지게 되었습니다:  
- **`messages`**: 입력 메시지  
- **`language`**: 언어 설정  

이 변경사항을 반영하기 위해 애플리케이션의 **상태(state)**를 업데이트해야 합니다.

In [19]:
from typing import Sequence
from langchain_core.messages import BaseMessage  # 메시지의 기본 클래스
from langgraph.graph.message import add_messages  # 메시지를 상태에 추가하기 위한 도구
from typing_extensions import Annotated, TypedDict  # 타입 힌팅 및 주석을 위해 사용

# 새로운 상태 스키마에 'messages'와 'language'를 추가합니다.
class State(TypedDict):
    """
    상태(State) 클래스
    - messages: 입력 및 출력 메시지를 저장합니다.
    - language: 언어 설정을 저장합니다.
    """
    # 메시지 시퀀스를 정의하고 add_messages 도구를 사용하여 관리합니다.
    messages: Annotated[Sequence[BaseMessage], add_messages]

    # 언어 설정을 위한 문자열 필드
    language: str

# 상태 스키마(State Schema)로 'State'를 사용하여 그래프를 정의합니다.
workflow = StateGraph(state_schema=State)

# 모델 호출 함수 정의
def call_model(state: State):
    """
    주어진 상태(State)를 기반으로 프롬프트를 생성하고 모델을 호출합니다.

    Args:
        state (State): 현재 상태 (메시지 및 언어 포함)

    Returns:
        dict: 모델의 응답을 메시지로 반환합니다.
    """
    # 상태에서 프롬프트 생성.
    prompt = prompt_template.invoke(state)

    # 생성된 프롬프트를 모델에 전달하고 응답을 받습니다.
    response = model.invoke(prompt)

    return {"messages": [response]}


workflow.add_edge(START, "model")

workflow.add_node("model", call_model)

# MemorySaver를 사용하여 상태 및 메시지 기록을 저장합니다.
memory = MemorySaver()

# 그래프를 메모리 체크포인트와 함께 컴파일합니다.
app = workflow.compile(checkpointer=memory)

In [20]:
#  'thread_id'를 사용하여 특정 대화 스레드를 식별
config = {"configurable": {"thread_id": "abc456"}}

query = "안녕, 나는 길동이야."  # 사용자의 메시지

language = "일본어"

# HumanMessage 객체를 메시지 목록에 추가
input_messages = [HumanMessage(query)]

# 애플리케이션 호출
output = app.invoke(
    {"messages": input_messages, "language": language},  # 메시지와 언어 전달
    config,   # 추가 설정 전달 (thread_id 포함)
)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

こんにちは、ギルドンさん！お元気ですか？何かお手伝いできることがあれば教えてください。


전체 상태가 유지되므로 변경이 필요하지 않으면 `language`와 같은 매개변수를 생략할 수 있습니다.

In [21]:
query = "내 이름이 뭐였지?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

あなたの名前はギルドンさんです。何か他にお手伝いできることがありますか？


내부에서 무슨 일이 일어나고 있는지 이해하는 데 도움이 되도록 [LangSmith 추적](https://smith.langchain.com/o/351c6cd9-1396-5c74-9478-1ee6a22a6433/projects/p/acec9d4d-4978-4597-adff-789cd42e200f?timeModel=%7B%22duration%22%3A%227d%22%7D)을 확인하세요.

### 대화 이력 관리  

챗봇을 구축할 때 중요한 개념 중 하나는 **대화 이력 관리**입니다. 이력이 관리되지 않으면 메시지 목록이 무한정으로 증가해 LLM의 **컨텍스트 윈도우를 초과할 가능성**이 있습니다. 따라서 전달하는 메시지의 크기를 제한하는 단계를 추가하는 것이 중요합니다.  

**중요한 점은, 이 단계는 프롬프트 템플릿 이전에, 그러나 메시지 이력을 불러온 이후에 수행해야 한다는 것입니다.**  

이를 위해 프롬프트 앞단에 `messages` 키를 적절히 수정하는 단계를 추가한 후, 이 새 체인을 **Message History 클래스**로 감싸면 됩니다.  

**LangChain**에는 메시지 목록을 관리하기 위한 몇 가지 내장 도구가 제공됩니다. 여기서는 trim_messages 헬퍼를 사용하여 모델에 전달하는 메시지 수를 줄일 것입니다.  

`trim_messages`를 사용하면 보유할 토큰 수, 시스템 메시지를 항상 유지할지 여부, 부분 메시지를 허용할지 여부와 같은 매개변수를 지정할 수 있습니다.

In [22]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

# 메시지 이력을 다듬기 위한 트리머(trimmer) 설정
trimmer = trim_messages(
    max_tokens=65,            # 최대 토큰 수를 65로 제한
    strategy="last",              # 최근 메시지부터 우선적으로 유지
    token_counter=model,  # 토큰 계산에 사용할 모델 (예: LLM 모델 객체)
    include_system=True,  # 시스템 메시지는 항상 포함
    allow_partial=False,     # 부분 메시지는 허용하지 않음
    start_on="human",       # 사람이 보낸 메시지부터 시작
)

# 메시지 이력 예제
messages = [
    SystemMessage(content="너는 도움이 되는 조수야."),    # 시스템 메시지 (기본 지시사항)
    HumanMessage(content="안녕! 내 이름은 길동이야."),         # 사용자의 첫 번째 메시지
    AIMessage(content="안녕하세요."),                                        # AI의 응답
    HumanMessage(content="난 바닐라 아이스크림을 좋아해"),  # 사용자의 두 번째 메시지
    AIMessage(content="멋져요."),                                        # AI의 응답
    HumanMessage(content="2 + 2는 얼마야?"),                   # 사용자의 세 번째 메시지
    AIMessage(content="4입니다."),                                        # AI의 응답
    HumanMessage(content="고마워!"),                            # 사용자의 네 번째 메시지
    AIMessage(content="천만에요!"),                                 # AI의 응답
    HumanMessage(content="재미있었어?"),                    # 사용자의 다섯 번째 메시지
    AIMessage(content="예!"),                                            # AI의 응답
]

# 트리머를 사용하여 메시지 이력을 다듬기
# 설정된 조건에 따라 오래된 메시지 또는 불필요한 메시지가 제거됨
trimmed_messages = trimmer.invoke(messages)

# 다듬어진 메시지 출력 (디버깅 또는 확인용)
print(trimmed_messages)

[SystemMessage(content='너는 도움이 되는 조수야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='고마워!', additional_kwargs={}, response_metadata={}), AIMessage(content='천만에요!', additional_kwargs={}, response_metadata={}), HumanMessage(content='재미있었어?', additional_kwargs={}, response_metadata={}), AIMessage(content='예!', additional_kwargs={}, response_metadata={})]


이를 체인에서 사용하려면 프롬프트에 `messages` 입력을 전달하기 전에 트리머를 실행하기만 하면 됩니다.

In [24]:
from langgraph.graph import START, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver

# 상태(State) 그래프 워크플로우 정의
workflow = StateGraph(state_schema=State)

# 모델 호출 함수 정의
def call_model(state: State):
    # 1. # 메시지 이력을 트리밍하여 토큰 수 제한
    trimmed_messages = trimmer.invoke(state["messages"])

    # 2. 프롬프트 템플릿 적용
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )

    # 3. LLM 호출
    response = model.invoke(prompt)

    # 4. 응답 메시지 반환
    return {"messages": [response]}  # 모델의 응답을 메시지 형태로 반환

# 워크플로우 노드 및 엣지 추가
workflow.add_edge(START, "model")  # START 상태에서 'model' 노드로 이동
workflow.add_node("model", call_model)  # 'model' 노드에 call_model 함수를 연결

# 상태를 저장 및 복원할 메모리 체크포인터 생성
memory = MemorySaver()

# 워크플로우 컴파일
app = workflow.compile(checkpointer=memory)  # 메모리 체크포인터를 사용해 워크플로우를 컴파일

이제 우리가 모델에게 우리의 이름을 물어보려고 하면, 우리가 채팅 기록의 해당 부분을 잘라냈기 때문에 모델이 우리의 이름을 알 수 없을 것입니다.

In [25]:
config = {"configurable": {"thread_id": "abc567"}}
query = "내 이름이 뭐야?"
language = "일본어"

# highlight-next-line
input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

죄송하지만, 당신의 이름은 알 수 없습니다. 이름을 말씀해 주시면 기억할 수 있어요!


하지만 최근 몇 개의 메시지에 있는 정보에 대해 묻는다면, 그것은 다음을 기억합니다:

In [26]:
config = {"configurable": {"thread_id": "abc456"}}
query = "내 이름이 뭐야?"
language = "일본어"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

죄송하지만, 당신의 이름은 알 수 없습니다. 하지만 당신이 원하시면 이름을 말씀해 주시면 좋겠습니다!


LangSmith를 살펴보면 [LangSmith 추적](https://smith.langchain.com/o/351c6cd9-1396-5c74-9478-1ee6a22a6433/projects/p/acec9d4d-4978-4597-adff-789cd42e200f?timeModel=%7B%22duration%22%3A%227d%22%7D)에서 정확히 무슨 일이 일어나고 있는지 확인할 수 있습니다.

## **스트리밍 (Streaming)**  

이제 우리는 작동하는 챗봇을 만들었습니다. 그러나 챗봇 애플리케이션의 사용자 경험(UX)에서 **매우 중요한 요소** 중 하나는 **스트리밍(Streaming)**입니다.  

LLM은 때때로 응답을 생성하는 데 시간이 걸릴 수 있습니다. 따라서 사용자 경험을 개선하기 위해 대부분의 애플리케이션은 **생성되는 각 토큰을 실시간으로 스트리밍**하여 사용자에게 보여줍니다. 이렇게 하면 사용자는 응답의 진행 상황을 실시간으로 확인할 수 있습니다.  

사실, 이 작업은 매우 간단합니다!  

기본적으로 LangGraph 애플리케이션에서 `.stream`은 애플리케이션 단계(예: 모델 응답 단계)를 스트리밍합니다. 그러나 `stream_mode="messages"`를 설정하면 **출력 토큰을 실시간으로 스트리밍**할 수 있습니다.  

In [27]:
from langchain_core.messages import HumanMessage, AIMessage

# 스트리밍 설정을 위한 구성(config) 정의
config = {"configurable": {"thread_id": "abc789"}}

# 사용자 입력 메시지 및 언어 설정
query = "안녕하세요, 제 이름은 토드입니다. 농담 하나 해주세요."  # 사용자 입력 메시지
language = "한국어"  # 대화 언어 설정

# 입력 메시지를 HumanMessage 객체로 변환
input_messages = [HumanMessage(query)]

# LangChain 애플리케이션 스트리밍 시작
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},  # 입력 메시지와 언어를 전달
    config,  # 구성 정보 전달
    stream_mode="messages",  # 메시지 단위로 토큰을 스트리밍
):
    # AIMessage 객체만 필터링하여 출력
    if isinstance(chunk, AIMessage):
        print(chunk.content, end="|")  # AI 응답 메시지를 실시간으로 출력 (구분자는 '|')

|안|녕하세요|,| 토|드|!| 농|담| 하나| 드|릴|게|요|.

퓨터|는| 바|다|를| 좋아|할|까요|?

냐|하면| 많은| "|파|도|"(|code|)가| 있기| 때문|이|죠|!| 

나요|?| 또| 다른| 농|담|이| 필요|하|시면| 말씀|해| 주세요|!||